In [1]:
import math
import cmath
import sys
import traceback
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp
from scipy import stats
from scipy.stats import t
from sklearn import model_selection
import matplotlib.pyplot as plt

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

# Mendefinisikan Konstruktor  dan Fungsi untuk Algoritma Perhitungan PCC

In [2]:
class CorrelationCoefficient:
    r = []
    t = []
    pval = []
    threshold = 0.05
    n = None
    def compute_r(self, x, y):
        """ R computation
        :param  list  x: 1st list of random variables
        :param  list  y: 2nd list of random variables
        :return float r: correlation coefficient of X and Y
        """
        if type(x) != list:
            print("Argument(X) is not a list!")
            sys.exit()
        if type(y) != list:
            print("Argument(Y) is not a list!")
            sys.exit()
        if len(x) == 0:
            print("List(X) is none!")
            sys.exit()
        if len(y) == 0:
            print("List(Y) is none!")
            sys.exit()
        if len(x) != len(y):
            print("Argument list size is invalid!")
            sys.exit()
        try:
            mean_x, mean_y  = sum(x) / len(x), sum(y) / len(y)
            cov   = sum([(a - mean_x) * (b - mean_y) for a, b in zip(x, y)])
            var_x = sum([(a - mean_x) ** 2 for a in x])
            var_y = sum([(b - mean_y) ** 2 for b in y])
          
            return (cov / math.sqrt(var_x)) / math.sqrt(var_y)
        except Exception as e:
            raise
            
    def compute_t(self, n, r_coeff):
      
        try:
            if r_coeff < 1 :
                t = r_coeff * (cmath.sqrt(n - 2)/ cmath.sqrt(1 - r_coeff ** 2))
            else:
                t = 0
            return t
        except Exception as e:
            raise
            
    def r_df_after_compute(self, np_array_r, index_name, col_name):
        try:
            df = pd.DataFrame(np_array_r, index=index_name, columns=col_name)
            
            return df
        except Exception as e:
            raise 
            
    def t_df_after_compute(self, np_array_t, index_name, col_name):
        try:
            df = pd.DataFrame(np_array_t, index=index_name, columns=col_name, dtype="float")
            return df
        except Exception as e:
            raise 
            
    def pval_df_after_compute(self, np_array_pval, index_name, col_name):
        try:
            df = pd.DataFrame(np_array_pval, index=index_name, columns=col_name)
            return df
        except Exception as e:
            raise 
            
    def drop_features(self, dataset, threshold, df):
        try:
            col_corr = set()
            corr_matrix = dataset
            for i in range(len(df.columns)):
                for j in range (len(df.columns)) :
                      if corr_matrix.iloc[i, j] > threshold and corr_matrix.iloc[i, j] != 1 and corr_matrix.columns[j] not in col_corr : #THRESHOLD CHECK
                          colname = corr_matrix.columns[i]  #NAMA DARI FITUR/ KOLOM
                          col_corr.add(colname)
        
            return col_corr
        except Exception as e:
            raise 
            

# Mendefinisikan Konstruktor  dan Fungsi untuk Algoritma Perhitungan ANOVA

In [3]:
class Anova:
    SS = []
    df = []
    F = []
    pval = []
    threshold = 0.05
    
    def compute_SS(self, groups, values, col_name):
    
        """ Calculate the ANOVA by hand. While you would normally not do that, this function shows
        how the underlying values can be calculated.
        """
        try:
            # The "total sum-square" is the squared deviation from the mean
            ss_total = np.sum((values-values.mean())**2)
          
            
            # Calculate ss_treatment and  ss_error
            (ss_treatments, ss_error) = (0, 0)
            for val, group in groups:
                ss_error += sum((group[col_name] - group[col_name].mean())**2)
               
                ss_treatments += len(group) * (group[col_name].mean() - values.mean())**2
             
            return (ss_treatments, ss_error, ss_total)
          
       
        except Exception as e:
            raise
            
            
    def compute_df(self, groups, data):
        try:
            df_groups = len(groups)-1
            df_residuals = len(data)-len(groups)
           
        
            return (df_groups, df_residuals)
          
       
        except Exception as e:
            raise
            
            
    def compute_F(self, ss_treatments, ss_error, df_groups, df_residuals):
        try:
            F = (ss_treatments/df_groups) / (ss_error/df_residuals)
            df = stats.f(df_groups,df_residuals)
            p = df.sf(F)
           
             # print(('ANOVA-Results: F = {0}, and p<{1}'.format(F, p)))
            return (F, p)
          
       
        except Exception as e:
            raise
            
            
            
    def ss_df_after_compute(self, np_array_ss, index_name, col_name):
        try:
            df = pd.DataFrame(np_array_ss, index=index_name, columns=col_name)
            return df
        except Exception as e:
            raise 
            
            
    def degreeFreedom_df_after_compute(self, np_array_df, index_name, col_name):
        try:
            df = pd.DataFrame(np_array_df, index=index_name, columns=col_name)
            return df
        except Exception as e:
            raise 
            
            
    def f_pval_df_after_compute(self, np_array_f, index_name, col_name):
        try:
            df = pd.DataFrame(np_array_f, index=index_name, columns=col_name)
            return df
        except Exception as e:
            raise 
            
            
    def drop_features(self, dataset, threshold, df):
        try:
            col_anova = set()
            anova_dataset = dataset
            for i in range(len(df.columns)-1):
                    if anova_dataset.iloc[-1, i] > threshold : #THRESHOLD CHECK
                        colname = anova_dataset.columns[i]  #NAMA DARI FITUR/ KOLOM
                        col_anova.add(colname)
        
            return col_anova
        except Exception as e:
            raise 

# Mendefinisikan Konstruktor  dan Fungsi untuk Klasifikasi

In [4]:
class classifier :
    validation_size = 0.60
    seed = 7      
            
    def model(self, drop_features, data):
        try:
            new_df = pd.DataFrame(data)
            new_df.drop(drop_features, axis=1, inplace=True)
         
           
            
            X = new_df.values[:,:-1]
            Y = new_df.values[:,-1:]
      
            
            np.random.seed(classifier.seed)
            
            X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=classifier.validation_size, random_state=classifier.seed)
            
            #return new_df
            return (X_train, X_test, Y_train, Y_test)
        except Exception as e:
            raise 
            
            
    def KNN(self, X_train, X_test, Y_train, Y_test):
        try:
            from sklearn.neighbors import KNeighborsClassifier
            model_KNN = KNeighborsClassifier(n_neighbors = 5, weights = 'distance', p = 3)
            
            
            
            model_KNN.fit(X_train,Y_train.ravel())
            Y_pred = model_KNN.predict(X_test)
            
            print('1) Confusion Matrix for KNN Model:')
            confusion_mat = pd.crosstab(Y_test.flatten(), Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
            from IPython.display import display
            display(confusion_mat)
            
            from sklearn.metrics import classification_report
            cr = classification_report(Y_test, Y_pred, labels=None, target_names=None, sample_weight=None, digits=2)
            #print('2) Classification report:\n',cr)
            
            accuracy_train = model_KNN.score(X_train,Y_train)
            accuracy_test = model_KNN.score(X_test,Y_test)
            #print('3) Accuracy on train and test datasets:', "%.4f and %.4f" % (accuracy_train, accuracy_test))
           
            
            return (accuracy_train, accuracy_test)
        
        
        
            
        except Exception as e:
            raise 
            
            
    def DecisionTree(self, X_train, X_test, Y_train, Y_test):
        try:
           from sklearn.tree import DecisionTreeClassifier
           model_DT = DecisionTreeClassifier(min_samples_leaf = 3, random_state = classifier.seed)
           
           
           model_DT.fit(X_train,Y_train)
           Y_pred = model_DT.predict(X_test)
            
           print('1) Confusion Matrix for DT Model:')
           confusion_mat = pd.crosstab(Y_test.flatten(), Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
           from IPython.display import display
           display(confusion_mat)
            
           from sklearn.metrics import classification_report
           cr = classification_report(Y_test.flatten(), Y_pred, labels=None, target_names=None, sample_weight=None, digits=2)
           #print('2) Classification report:\n',cr)
            
           accuracy_train = model_DT.score(X_train,Y_train)
           accuracy_test = model_DT.score(X_test,Y_test)
           #print('3) Accuracy on train and test datasets:', "%.4f and %.4f" % (accuracy_train, accuracy_test))
           
           return (accuracy_train, accuracy_test)
       
        
        except Exception as e:
            raise 
            
            
    def SupportVectorMachine(self, X_train, X_test, Y_train, Y_test):
        try:
            from sklearn import svm
            model_SVC = svm.SVC(gamma = 'auto')
            
            
            model_SVC.fit(X_train,Y_train.ravel())
            Y_pred = model_SVC.predict(X_test)
            
            print('1) Confusion Matrix for SVC Model:')
            confusion_mat = pd.crosstab(Y_test.flatten(), Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
            from IPython.display import display
            display(confusion_mat)
            
            from sklearn.metrics import classification_report
            cr = classification_report(Y_test, Y_pred, labels=None, target_names=None, sample_weight=None, digits=2)
            #print('2) Classification report:\n',cr)
            
            accuracy_train = model_SVC.score(X_train,Y_train)
            accuracy_test = model_SVC.score(X_test,Y_test)
            #print('3) Accuracy on train and test datasets:', "%.4f and %.4f" % (accuracy_train, accuracy_test))
            
            return (accuracy_train, accuracy_test)
        
        
        except Exception as e:
            raise 
            
            
    def NaiveBayes(self, X_train, X_test, Y_train, Y_test):
        try:
            from sklearn.naive_bayes import GaussianNB
            model_NB = GaussianNB()
            
            
            model_NB.fit(X_train,Y_train.ravel())
            Y_pred = model_NB.predict(X_test)
            
            print('1) Confusion Matrix for NaiveBayes Model:')
            confusion_mat = pd.crosstab(Y_test.flatten(), Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
            from IPython.display import display
            display(confusion_mat)
            
            from sklearn.metrics import classification_report
            cr = classification_report(Y_test, Y_pred, labels=None, target_names=None, sample_weight=None, digits=2)
           #print('2) Classification report:\n',cr)
            
            accuracy_train = model_NB.score(X_train,Y_train)
            accuracy_test = model_NB.score(X_test,Y_test)
            #print('3) Accuracy on train and test datasets:', "%.4f and %.4f" % (accuracy_train, accuracy_test))
        
            return (accuracy_train, accuracy_test)
        
        
        except Exception as e:
            raise 
            
    def accuracy_df(self, np_array_df, index_name):
        try:
            df_accuracy = pd.DataFrame(np_array_df, index=index_name)
         
            return df_accuracy
        except Exception as e:
            raise 

# GRAPHIC USER INTERFACE

In [5]:
import tkinter as tk
import ttkbootstrap as ttk
from ttkbootstrap.constants import *

from ttkbootstrap import Style

# import filedialog module
from tkinter import filedialog
class mainApp() :
    def __init__ (self, root):
        self.label_file_explorer = None
        self.root = root
        self.frontPages(self.root)
       
    def frontPages(self, root):
        b1 = ttk.Button(root,padding="245 75 245 75", text="PCC", command = self.pccWindow)
        b1.place(x=0, y=0)
        b2 = ttk.Button(root,padding="200 75 200 75", text="ANOVA F-TEST", command =self.anovaWindow)
        b2.place(x=0, y=180)
        
    def pccWindow(self):
        self.closeMain()
        
     
        self.rootPCC = ttk.Window()
        self.rootPCC.configure(bg = "#78c2ad")
       
        backButton = ttk.Button(self.rootPCC, padding="50 20 50 20", text="Back", command = self.openMain)
        backButton.place(x=1150, y=50)
        
        self.label_file_explorer_pcc = ttk.Label(self.rootPCC, text="No File chosen")
        self.label_file_explorer_pcc.place(x=40, y=170, width=550,height=50)
        self.label_file_explorer_pcc.configure(anchor="center")
        
        browseButton = ttk.Button(self.rootPCC, padding="50 10 50 10", text="Browse", command = self.browseFilesPCC)
        browseButton.place(x=600, y=170)
        
        generateButton = ttk.Button(self.rootPCC, padding="50 10 50 10", text="Generate", command = self.generateButtonActionPCC)
        generateButton.place(x=600, y=225)
        
       
        self.label_file_explorer2_pcc = ttk.Label(self.rootPCC, background = "#78c2ad")
        self.label_file_explorer2_pcc.place(x=40, y=300, width=700,height=50)
        self.label_file_explorer2_pcc.configure(anchor="center")
       
        self.label_featuresLength_pcc = ttk.Label(self.rootPCC, background = "#78c2ad")
        self.label_featuresLength_pcc.place(x=40, y=360, width=550,height=50)
        self.label_featuresLength_pcc.configure(anchor="center")
        
        self.label_knn_pcc = ttk.Label(self.rootPCC, background = "#78c2ad")
        self.label_knn_pcc.place(x=40, y=450, width=550,height=50)
        self.label_knn_pcc.configure(anchor="center")
        
        self.label_dt_pcc = ttk.Label(self.rootPCC, background = "#78c2ad")
        self.label_dt_pcc.place(x=40, y=500, width=550,height=50)
        self.label_dt_pcc.configure(anchor="center")
        
        self.label_svm_pcc = ttk.Label(self.rootPCC, background = "#78c2ad")
        self.label_svm_pcc.place(x=40, y=550, width=550,height=50)
        self.label_svm_pcc.configure(anchor="center")
        
        self.label_nb_pcc = ttk.Label(self.rootPCC, background = "#78c2ad")
        self.label_nb_pcc.place(x=40, y=600, width=550,height=50)
        self.label_nb_pcc.configure(anchor="center")
        
        self.frame_pcc = ttk.Frame(self.rootPCC)
        self.frame_pcc.place(x=600, y=460)
        
        self.frame_heatmap_pcc = ttk.Frame(self.rootPCC)
        self.frame_heatmap_pcc.place(x=1000, y=315)
        
        self.rootPCC.geometry("1600x900")
        self.rootPCC.mainloop()
        
    def anovaWindow(self):

        self.closeMain()
        
        
        
        self.rootANOVA = ttk.Window()
        self.rootANOVA.configure(bg = "#78c2ad")
        
      
        
        backButton_ANOVA = ttk.Button(self.rootANOVA, padding="50 20 50 20", text="Back", command = self.openMain2)
        backButton_ANOVA.place(x=1150, y=50)
  
        
        self.label_file_explorer_anova = ttk.Label(self.rootANOVA, text="No File chosen")
        self.label_file_explorer_anova.place(x=40, y=170, width=550,height=50)
        self.label_file_explorer_anova.configure(anchor="center")
        
        browseButton2 = ttk.Button(self.rootANOVA, padding="50 10 50 10", text="Browse", command = self.browseFilesANOVA)
        browseButton2.place(x=600, y=170)
        
        generateButton2 = ttk.Button(self.rootANOVA, padding="50 10 50 10", text="Generate", command = self.generateButtonActionANOVA)
        generateButton2.place(x=600, y=225)
        
       
        self.label_file_explorer2_anova = ttk.Label(self.rootANOVA, background = "#78c2ad")
        self.label_file_explorer2_anova.place(x=40, y=300, width=550,height=50)
        self.label_file_explorer2_anova.configure(anchor="center")
       
        self.label_featuresLength_anova = ttk.Label(self.rootANOVA, background = "#78c2ad")
        self.label_featuresLength_anova.place(x=40, y=350, width=550,height=50)
        self.label_featuresLength_anova.configure(anchor="center")
        
        self.label_knn_anova = ttk.Label(self.rootANOVA, background = "#78c2ad")
        self.label_knn_anova.place(x=40, y=450, width=550,height=50)
        self.label_knn_anova.configure(anchor="center")
        
        self.label_dt_anova = ttk.Label(self.rootANOVA, background = "#78c2ad")
        self.label_dt_anova.place(x=40, y=500, width=550,height=50)
        self.label_dt_anova.configure(anchor="center")
        
        self.label_svm_anova = ttk.Label(self.rootANOVA, background = "#78c2ad")
        self.label_svm_anova.place(x=40, y=550, width=550,height=50)
        self.label_svm_anova.configure(anchor="center")
        
        self.label_nb_anova = ttk.Label(self.rootANOVA, background = "#78c2ad")
        self.label_nb_anova.place(x=40, y=600, width=550,height=50)
        self.label_nb_anova.configure(anchor="center")
        
        
        self.frame_anova = ttk.Frame(self.rootANOVA)
        self.frame_anova.place(x=600, y=460)
        
        self.frameDf_anova = ttk.Frame(self.rootANOVA)
        self.frameDf_anova.place(x=1000, y=150, width=500,height=600)
       
        self.rootANOVA.geometry("1600x900")
        self.rootANOVA.mainloop()
        
    def browseFilesANOVA(self):
        
        self.filenameANOVA = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("csv files",
                                                        "*.csv*"),
                                                       ("all files",
                                                        "*.*")))
      
        # Change label contents
        self.label_file_explorer_anova.configure(text=self.filenameANOVA)
    def browseFilesPCC(self):
        
        self.filenamePCC = filedialog.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("csv files",
                                                        "*.csv*"),
                                                       ("all files",
                                                        "*.*")))
      
        # Change label contents
        self.label_file_explorer_pcc.configure(text=self.filenamePCC)
        
    def closeMain(self):
        self.root.withdraw()
        
    def closePCC(self):
        self.rootPCC.withdraw()
        
    def closeANOVA(self):
        self.rootANOVA.withdraw()
        
    def mainWindow(self):
        self.root.deiconify()
        
    def openMain(self):
        self.mainWindow()
        self.closePCC()
        
    def openMain2(self):
        self.mainWindow()
        self.closeANOVA()
        
    def generateButtonActionANOVA(self):
        self.data = pd.read_csv(self.filenameANOVA)
        self.df = pd.DataFrame(self.data)
        for i in range(len(self.df.columns)-1):
            values = self.df.iloc[:, i]

            category = self.df.columns[-1:].tolist()
            col_name = self.df.columns[i]

            groups = self.df.groupby(category)

            obj = Anova()
            # PERHITUNGAN SS, df, F, P-VALUE
            SS_coeff = obj.compute_SS(groups, values, col_name)
            df_coeff = obj.compute_df(groups, self.data)
            F_coeff = obj.compute_F(SS_coeff[0], SS_coeff[1], df_coeff[0], df_coeff[1])


            # MEMASUKKAN PERHITUNGAN KEDALAM LIST
            obj.SS.append(SS_coeff)
            obj.df.append(df_coeff)
            obj.F.append(F_coeff)
            
        np_array_SS = np.array(obj.SS).T
        np_array_df = np.array(obj.df).T
        np_array_f = np.array(obj.F).T
        
        index_name_SS = ['SS Treatments', 'SS Error', 'SS Totals']
        index_name_df = ['df GROUP', 'df RESIDUALS']
        index_name_F = ['F', 'P-VAL']
        col_name = self.df.columns.drop('Results')


        #MENGUBAH NUMPY ARRAY MENJADI DATAFRAME UNTUK MENYIMPAN HASIL ANOVA
        SS_result = obj.ss_df_after_compute(np_array_SS, index_name_SS, col_name)
        df_result = obj.degreeFreedom_df_after_compute(np_array_df, index_name_df, col_name)
        F_PVAL_result = obj.f_pval_df_after_compute(np_array_f, index_name_F, col_name)
        drop_features = obj.drop_features(F_PVAL_result, obj.threshold, self.df)
        
        #print(drop_features)
        
        #print(len(drop_features))
        self.label_file_explorer2_anova.configure(text=drop_features)
        self.label_featuresLength_anova.configure(text=len(drop_features))
        
        objClassifier = classifier()
        model = objClassifier.model(drop_features, self.data)
        
        
        KNN = objClassifier.KNN(model[0], model[1], model[2], model[3])
        DT = objClassifier.DecisionTree(model[0], model[1], model[2], model[3])
        SVM = objClassifier.SupportVectorMachine(model[0], model[1], model[2], model[3])
        NB = objClassifier.NaiveBayes(model[0], model[1], model[2], model[3])
        
        self.label_knn_anova.configure(text='KNN : '+str(KNN[1]))
        self.label_dt_anova.configure(text="DECISION TREE : " +str(DT[1]))
        self.label_svm_anova.configure(text="SVM : " +str(SVM[1]))
        self.label_nb_anova.configure(text="NAIVE BAYES : " +str(NB[1]))
        
        self.index_name_accuracy_anova = ['KNN', 'DT', 'SVM', 'NB']
        self.rowAccuracy_anova = []
        self.rowAccuracy_anova.append(KNN[1])
        self.rowAccuracy_anova.append(DT[1])
        self.rowAccuracy_anova.append(SVM[1])
        self.rowAccuracy_anova.append(NB[1])

        self.df_acc_anova = objClassifier.accuracy_df(self.rowAccuracy_anova, self.index_name_accuracy_anova)
        
        
        f, ax = plt.subplots(figsize=(5, 4))
        bar = sns.barplot(x=self.df_acc_anova.index, y=self.df_acc_anova[0])
        
        
        canvas = FigureCanvasTkAgg(f, master=self.frame_anova)
        canvas.draw()
        canvas.get_tk_widget().pack()
        
        text = tk.Text(self.frameDf_anova)
        text.insert(tk.END, str(F_PVAL_result.transpose()))
        text.pack()
        
    def generateButtonActionPCC(self):
        self.data_pcc = pd.read_csv(self.filenamePCC)
        self.df_pcc = pd.DataFrame(self.data_pcc)
        self.df_pcc.drop('Results', axis = 1, inplace=True)
        obj = CorrelationCoefficient()
        obj.n = len(self.df_pcc)
        for i in range(len(self.df_pcc.columns)):
            x = self.df_pcc.iloc[:, i].tolist()
            col_r = []
            col_t = []
            col_pval = []
            for j in range (len(self.df_pcc.columns)):
                y = self.df_pcc.iloc[:, j].tolist()
                

                # PERHITUNGAN R, T-TEST, P-VALUE
                r_coeff = obj.compute_r(x, y)
                t_test = obj.compute_t(obj.n, r_coeff)
                p_val = t.sf(np.abs(t_test), obj.n-1)*2 
                
                # MEMASUKKAN PERHITUNGAN KEDALAM LIST
                col_r.append(r_coeff)
                col_t.append(t_test)
                col_pval.append(p_val)
            
          
            
            obj.r.append(col_r)
            obj.t.append(col_t)
            obj.pval.append(col_pval)
            
            
            np_array_r = np.array(obj.r)
            np_array_ttest = np.array(obj.t)
            np_array_pval = np.array(obj.pval)
            
            
        r_df = obj.r_df_after_compute(np_array_r, self.df_pcc.columns, self.df_pcc.columns)
        t_df = obj.t_df_after_compute(np_array_ttest, self.df_pcc.columns, self.df_pcc.columns)
        pval_df = obj.pval_df_after_compute(np_array_pval, self.df_pcc.columns, self.df_pcc.columns)

        objClassifier = classifier()
        drop_features = obj.drop_features(pval_df, obj.threshold, self.df_pcc)
        #print(drop_features)
        #print(len(drop_features))
        self.label_file_explorer2_pcc.configure(text=drop_features)
        self.label_featuresLength_pcc.configure(text=len(drop_features))
        model = objClassifier.model(drop_features, self.data_pcc)
        
        KNN = objClassifier.KNN(model[0], model[1], model[2], model[3])
        DT = objClassifier.DecisionTree(model[0], model[1], model[2], model[3])
        SVM = objClassifier.SupportVectorMachine(model[0], model[1], model[2], model[3])
        NB = objClassifier.NaiveBayes(model[0], model[1], model[2], model[3])
        
        self.label_knn_pcc.configure(text='KNN : '+str(KNN[1]))
        self.label_dt_pcc.configure(text="DECISION TREE : " +str(DT[1]))
        self.label_svm_pcc.configure(text="SVM : " +str(SVM[1]))
        self.label_nb_pcc.configure(text="NAIVE BAYES : " +str(NB[1]))
        
        self.index_name_accuracy_pcc = ['KNN', 'DT', 'SVM', 'NB']
        self.rowAccuracy_pcc = []
        self.rowAccuracy_pcc.append(KNN[1])
        self.rowAccuracy_pcc.append(DT[1])
        self.rowAccuracy_pcc.append(SVM[1])
        self.rowAccuracy_pcc.append(NB[1])

        self.df_acc_pcc = objClassifier.accuracy_df(self.rowAccuracy_pcc, self.index_name_accuracy_pcc)
        
        f, ax = plt.subplots(figsize=(5, 4))
        bar = sns.barplot(x=self.df_acc_pcc.index, y=self.df_acc_pcc[0])
        
        
        f1,ax1 = plt.subplots(figsize=(6, 6))
        heatmap = sns.heatmap(pval_df, xticklabels=True, yticklabels=True)
        
        canvas_barplot = FigureCanvasTkAgg(f, master=self.frame_pcc)
        canvas_barplot.draw()
        canvas_barplot.get_tk_widget().pack()
        
        canvas_heatmap = FigureCanvasTkAgg(f1, master=self.frame_heatmap_pcc)
        canvas_heatmap.draw()
        canvas_heatmap.get_tk_widget().pack()


In [ ]:

root = ttk.Window()
style = ttk.Style("minty")
style.configure('TButton', font=('Banschrift', 11))
app = mainApp(root)
root.geometry("525x360")
root.mainloop()       


D:\Anaconda\lib\site-packages\pandas\core\internals\construction.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  values = values.astype(dtype)


1) Confusion Matrix for KNN Model:


Predicted,Depressed,No,All
Actual,,,
Depressed,48,80,128
No,286,432,718
All,334,512,846


1) Confusion Matrix for DT Model:


Predicted,Depressed,No,All
Actual,,,
Depressed,20,108,128
No,105,613,718
All,125,721,846


...

Predicted,No,All
Actual,,
Depressed,128,128
No,718,718
All,846,846


1) Confusion Matrix for NaiveBayes Model:


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predicted,No,All
Actual,,
Depressed,128,128
No,718,718
All,846,846


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
